In [52]:
import requests
import json 

# GET 
def rest_get(url):
    response = requests.get(url) 
    # print(str(response.status_code) + " | " + response.text) 
    return json.loads(response.text)

def rest_get_raw(url, header):
    response = requests.get(url, headers=header) 
    return json.loads(response.text)

# POST (JSON) 
def rest_post(url, token, data):
    header = {'X-Auth-Token': token, 'Content-Type': 'application/json; chearset=utf-8'}
    response = requests.post(url, data=json.dumps(data), headers=header) 
    # print(str(response.status_code) + " | " + response.text)
    return json.loads(response.text)

def rest_post_raw(url, header, data):
    response = requests.post(url, data=json.dumps(data), headers=header) 
    return json.loads(response.text)

def rest_put_raw(url, header, data):
    response = requests.put(url, data=json.dumps(data), headers=header)
#     print(str(response.status_code) + " | " + response.text)
    return json.loads(response.text)

In [53]:
# get location info
def get_location(auth_key):
    url = f"{BASE_URL}/locations"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    location = rest_get_raw(url, header)
    return location

In [54]:
# get truck info
def get_truck(auth_key):
    url = f"{BASE_URL}/trucks"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    truck = rest_get_raw(url, header)
    return truck

In [55]:
def put_simulate(auth_key, data):
    url = f"{BASE_URL}/simulate"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    simulate = rest_put_raw(url, header, data)
    return simulate

In [56]:
def get_score(auth_key):
    url = f"{BASE_URL}/score"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    score = rest_get_raw(url, header)
    
    return score

In [557]:
BASE_URL = 'https://kox947ka1a.execute-api.ap-northeast-2.amazonaws.com/prod/users'
X_AUTH_TOKEN = "ed99770e51cc89c9937ec8ce68eaeb2a"

In [558]:
# start api
url = f"{BASE_URL}/start"
data = {"problem": 1}
start = rest_post(url, X_AUTH_TOKEN, data)
auth_key = start["auth_key"]

In [559]:
location = get_location(auth_key)
truck = get_truck(auth_key)
print(location)
print()
print(truck)

{'locations': [{'id': 0, 'located_bikes_count': 4}, {'id': 1, 'located_bikes_count': 4}, {'id': 2, 'located_bikes_count': 4}, {'id': 3, 'located_bikes_count': 4}, {'id': 4, 'located_bikes_count': 4}, {'id': 5, 'located_bikes_count': 4}, {'id': 6, 'located_bikes_count': 4}, {'id': 7, 'located_bikes_count': 4}, {'id': 8, 'located_bikes_count': 4}, {'id': 9, 'located_bikes_count': 4}, {'id': 10, 'located_bikes_count': 4}, {'id': 11, 'located_bikes_count': 4}, {'id': 12, 'located_bikes_count': 4}, {'id': 13, 'located_bikes_count': 4}, {'id': 14, 'located_bikes_count': 4}, {'id': 15, 'located_bikes_count': 4}, {'id': 16, 'located_bikes_count': 4}, {'id': 17, 'located_bikes_count': 4}, {'id': 18, 'located_bikes_count': 4}, {'id': 19, 'located_bikes_count': 4}, {'id': 20, 'located_bikes_count': 4}, {'id': 21, 'located_bikes_count': 4}, {'id': 22, 'located_bikes_count': 4}, {'id': 23, 'located_bikes_count': 4}, {'id': 24, 'located_bikes_count': 4}]}

{'trucks': [{'id': 0, 'loaded_bikes_count':

In [560]:
static = [0 for i in range(25)]

In [561]:
from pprint import pprint

### 반복

In [562]:
while True:

    command = [{"truck_id": i, "command": []} for i in range(5)]

    # get info
    location = get_location(auth_key)
    truck = get_truck(auth_key)

    # check status
    MAX_LOC = []
    MIN_LOC = []
    MEAN = round(sum(list(map(lambda x: x['located_bikes_count'],location['locations'])))/len(location['locations']))
    for loc in location['locations']:
        if loc["located_bikes_count"] > MEAN:
            MAX_LOC.append((loc["id"], loc["located_bikes_count"]))
        elif loc["located_bikes_count"] < MEAN:
            MIN_LOC.append((loc["id"], loc["located_bikes_count"]))

    MAX_LOC.sort(key=lambda x : -x[1])
    MIN_LOC.sort(key=lambda x : x[1])
    
    # make command
    if len(MAX_LOC) != 0 and len(MIN_LOC) != 0:

        # 이미 bike 싣고 있음
        loaded_trucks = list(filter(lambda tr : tr['loaded_bikes_count'] >= 2, truck["trucks"]))

        # bike가 없는 경우
        unloaded_trucks = list(filter(lambda tr : tr['loaded_bikes_count'] < 2, truck["trucks"]))
        tmp_dict = {}
        for tr in unloaded_trucks:
            tr, loc = find_max_loc(tr, MAX_LOC, location, MEAN)
            if tr == -1:
                break

            load_num = location['locations'][loc]['located_bikes_count'] - MEAN
            command[tr['id']]["command"] += truck_move_plan(truck['trucks'][tr['id']]['location_id'], loc) \
                           + [5 for i in range(load_num)]
            truck['trucks'][tr['id']] = {'id': tr['id'], 'loaded_bikes_count': load_num, 'location_id': loc}
            location['locations'][loc]['located_bikes_count'] -= load_num
            loaded_trucks.append(truck['trucks'][tr['id']])


        # 배송    
        for dst in MIN_LOC[:5]:
            
            if len(loaded_trucks) == 0:
                break
                
            # 운행
            dst = dst[0]
            tr, v = find_truck(loaded_trucks, dst)
            src = tr['location_id']
            plan = truck_move_plan(src, dst)

            loaded_trucks.remove(tr)
            load_num = tr['loaded_bikes_count']
            command[tr['id']]["command"] += plan + [6 for i in range(load_num)]
            location['locations'][dst]['located_bikes_count'] += load_num
            truck['trucks'][tr['id']] = {'id': tr['id'], 'loaded_bikes_count': 0, 'location_id': dst}
            # 하차

        for cmd in command:
            cmd['command'] += [0 for i in range(10 - len(cmd['command']))]
            
    result = put_simulate(auth_key, {"commands": command})

    flag = result["status"]

    if flag == "finished":
        score = get_score(auth_key)
        print(score)
        break

200 | {"status":"ready","time":1,"failed_requests_count":"0.0","distance":"0.0"}
200 | {"status":"ready","time":2,"failed_requests_count":"0.0","distance":"0.0"}
200 | {"status":"ready","time":3,"failed_requests_count":"0.0","distance":"0.0"}
200 | {"status":"ready","time":4,"failed_requests_count":"0.0","distance":"0.0"}
200 | {"status":"ready","time":5,"failed_requests_count":"0.0","distance":"0.0"}
200 | {"status":"ready","time":6,"failed_requests_count":"0.0","distance":"0.0"}
200 | {"status":"ready","time":7,"failed_requests_count":"0.0","distance":"0.0"}
200 | {"status":"ready","time":8,"failed_requests_count":"0.0","distance":"0.0"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 0}, 0)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 0}, 1)
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 0}, 4)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 0}, 6)
({'id': 4, 'loaded_bikes_count': 0, 'location_id': 0}, 3)
200 | {"status":"ready","time":9,"failed_requests_count":"0.0"

200 | {"status":"ready","time":53,"failed_requests_count":"3.0","distance":"27.9"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 2}, 7)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 21}, 20)
({'id': 2, 'loaded_bikes_count': 1, 'location_id': 1}, 0)
({'id': 4, 'loaded_bikes_count': 0, 'location_id': 24}, 19)
200 | {"status":"ready","time":54,"failed_requests_count":"3.0","distance":"28.4"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 6}, 5)
({'id': 1, 'loaded_bikes_count': 1, 'location_id': 20}, 24)
({'id': 4, 'loaded_bikes_count': 1, 'location_id': 19}, 9)
200 | {"status":"ready","time":55,"failed_requests_count":"3.0","distance":"29.9"}
({'id': 1, 'loaded_bikes_count': 1, 'location_id': 21}, 13)
200 | {"status":"ready","time":56,"failed_requests_count":"4.0","distance":"30.7"}
200 | {"status":"ready","time":57,"failed_requests_count":"4.0","distance":"30.7"}
200 | {"status":"ready","time":58,"failed_requests_count":"4.0","distance":"30.7"}
({'id': 0, 'loaded_bikes_coun

200 | {"status":"ready","time":100,"failed_requests_count":"4.0","distance":"60.3"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 6}, 17)
(-1, -1)
200 | {"status":"ready","time":101,"failed_requests_count":"4.0","distance":"60.8"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 13}, 3)
({'id': 1, 'loaded_bikes_count': 1, 'location_id': 3}, 6)
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 11}, 10)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 10}, 24)
(-1, -1)
200 | {"status":"ready","time":102,"failed_requests_count":"4.0","distance":"62.7"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 23}, 23)
({'id': 2, 'loaded_bikes_count': 1, 'location_id': 10}, 8)
(-1, -1)
200 | {"status":"ready","time":103,"failed_requests_count":"4.0","distance":"63.4"}
200 | {"status":"ready","time":104,"failed_requests_count":"4.0","distance":"63.4"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 21}, 7)
(-1, -1)
200 | {"status":"ready","time":105,"failed_requests_count":"4.0","dist

200 | {"status":"ready","time":140,"failed_requests_count":"8.0","distance":"87.9"}
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 6}, 6)
200 | {"status":"ready","time":141,"failed_requests_count":"8.0","distance":"88.0"}
200 | {"status":"ready","time":142,"failed_requests_count":"8.0","distance":"88.0"}
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 20}, 20)
200 | {"status":"ready","time":143,"failed_requests_count":"8.0","distance":"88.7"}
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 19}, 14)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 2}, 2)
200 | {"status":"ready","time":144,"failed_requests_count":"8.0","distance":"89.3"}
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 15}, 15)
({'id': 3, 'loaded_bikes_count': 1, 'location_id': 2}, 16)
200 | {"status":"ready","time":145,"failed_requests_count":"8.0","distance":"91.6"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 1}, 1)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 3}, 2)
({'id': 2, 'loaded_bikes_

200 | {"status":"ready","time":183,"failed_requests_count":"11.0","distance":"121.3"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 6}, 17)
(-1, -1)
200 | {"status":"ready","time":184,"failed_requests_count":"11.0","distance":"121.9"}
200 | {"status":"ready","time":185,"failed_requests_count":"11.0","distance":"121.9"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 6}, 4)
(-1, -1)
200 | {"status":"ready","time":186,"failed_requests_count":"11.0","distance":"122.8"}
200 | {"status":"ready","time":187,"failed_requests_count":"11.0","distance":"122.8"}
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 0}, 4)
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 3}, 7)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 13}, 9)
(-1, -1)
200 | {"status":"ready","time":188,"failed_requests_count":"11.0","distance":"124.8"}
200 | {"status":"ready","time":189,"failed_requests_count":"11.0","distance":"124.8"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 10}, 19)
(-1, -1)
200 | {"st

...

200 | {"status":"ready","time":518,"failed_requests_count":"49.0","distance":"349.8"}
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 24}, 24)
200 | {"status":"ready","time":519,"failed_requests_count":"49.0","distance":"350.0"}
({'id': 2, 'loaded_bikes_count': 1, 'location_id': 24}, 24)
({'id': 4, 'loaded_bikes_count': 0, 'location_id': 4}, 4)
200 | {"status":"ready","time":520,"failed_requests_count":"50.0","distance":"350.1"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 15}, 15)
({'id': 4, 'loaded_bikes_count': 1, 'location_id': 4}, 3)
200 | {"status":"ready","time":521,"failed_requests_count":"50.0","distance":"350.3"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 15}, 19)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 9}, 9)
200 | {"status":"ready","time":522,"failed_requests_count":"50.0","distance":"351.4"}
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 21}, 21)
({'id': 3, 'loaded_bikes_count': 1, 'location_id': 9}, 7)
200 | {"status":"ready","time":523,"fa

200 | {"status":"ready","time":557,"failed_requests_count":"57.0","distance":"377.9"}
200 | {"status":"ready","time":558,"failed_requests_count":"57.0","distance":"377.9"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 15}, 15)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 13}, 11)
({'id': 2, 'loaded_bikes_count': 1, 'location_id': 20}, 10)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 24}, 24)
({'id': 4, 'loaded_bikes_count': 0, 'location_id': 7}, 6)
200 | {"status":"ready","time":559,"failed_requests_count":"57.0","distance":"379.1"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 22}, 22)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 12}, 13)
({'id': 3, 'loaded_bikes_count': 1, 'location_id': 24}, 4)
({'id': 4, 'loaded_bikes_count': 0, 'location_id': 13}, 16)
200 | {"status":"ready","time":560,"failed_requests_count":"57.0","distance":"380.1"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 22}, 15)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 18}, 6)


200 | {"status":"ready","time":598,"failed_requests_count":"59.0","distance":"405.0"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 1}, 3)
({'id': 1, 'loaded_bikes_count': 1, 'location_id': 0}, 20)
({'id': 2, 'loaded_bikes_count': 1, 'location_id': 12}, 21)
(-1, -1)
200 | {"status":"ready","time":599,"failed_requests_count":"59.0","distance":"407.3"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 5}, 0)
(-1, -1)
200 | {"status":"ready","time":600,"failed_requests_count":"59.0","distance":"408.2"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 19}, 8)
(-1, -1)
200 | {"status":"ready","time":601,"failed_requests_count":"59.0","distance":"408.7"}
200 | {"status":"ready","time":602,"failed_requests_count":"59.0","distance":"408.7"}
200 | {"status":"ready","time":603,"failed_requests_count":"59.0","distance":"408.7"}
200 | {"status":"ready","time":604,"failed_requests_count":"59.0","distance":"408.7"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 4}, 9)
(-1, -1)
200 | {"st

200 | {"status":"ready","time":637,"failed_requests_count":"62.0","distance":"430.1"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 1}, 1)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 4}, 10)
({'id': 2, 'loaded_bikes_count': 1, 'location_id': 9}, 7)
200 | {"status":"ready","time":638,"failed_requests_count":"62.0","distance":"431.1"}
200 | {"status":"ready","time":639,"failed_requests_count":"62.0","distance":"431.1"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 0}, 16)
({'id': 1, 'loaded_bikes_count': 0, 'location_id': 15}, 21)
200 | {"status":"ready","time":640,"failed_requests_count":"62.0","distance":"431.9"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 16}, 20)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 20}, 14)
200 | {"status":"ready","time":641,"failed_requests_count":"63.0","distance":"433.1"}
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 6}, 6)
(-1, -1)
200 | {"status":"ready","time":642,"failed_requests_count":"63.0","distance":"433.9"}
({'

200 | {"status":"ready","time":680,"failed_requests_count":"64.0","distance":"460.0"}
({'id': 0, 'loaded_bikes_count': 0, 'location_id': 14}, 14)
({'id': 1, 'loaded_bikes_count': 1, 'location_id': 5}, 5)
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 10}, 10)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 22}, 11)
(-1, -1)
200 | {"status":"ready","time":681,"failed_requests_count":"64.0","distance":"461.6"}
200 | {"status":"ready","time":682,"failed_requests_count":"64.0","distance":"461.6"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 14}, 12)
(-1, -1)
200 | {"status":"ready","time":683,"failed_requests_count":"64.0","distance":"462.1"}
({'id': 0, 'loaded_bikes_count': 1, 'location_id': 15}, 15)
({'id': 1, 'loaded_bikes_count': 1, 'location_id': 4}, 3)
({'id': 2, 'loaded_bikes_count': 0, 'location_id': 23}, 14)
({'id': 3, 'loaded_bikes_count': 0, 'location_id': 2}, 11)
({'id': 4, 'loaded_bikes_count': 1, 'location_id': 23}, 16)
200 | {"status":"ready","time":684,"failed_

In [531]:
# while True:

command = [{"truck_id": i, "command": []} for i in range(5)]

# get info
location = get_location(auth_key)
truck = get_truck(auth_key)
pprint(location)
pprint(truck)


# check status
# TRUCK_BIKE_MAX_NUM = 6
# TRUCK_BIKE_MIN_NUM = 2
MAX_LOC = []
MIN_LOC = []
MEAN = round(sum(list(map(lambda x: x['located_bikes_count'],location['locations'])))/len(location['locations']))
for loc in location['locations']:
    if loc["located_bikes_count"] > MEAN:
        MAX_LOC.append((loc["id"], loc["located_bikes_count"]))
    elif loc["located_bikes_count"] < MEAN:
        MIN_LOC.append((loc["id"], loc["located_bikes_count"]))
        
MAX_LOC.sort(key=lambda x : -x[1])
MIN_LOC.sort(key=lambda x : x[1])

{'locations': [{'id': 0, 'located_bikes_count': 4},
               {'id': 1, 'located_bikes_count': 4},
               {'id': 2, 'located_bikes_count': 4},
               {'id': 3, 'located_bikes_count': 4},
               {'id': 4, 'located_bikes_count': 4},
               {'id': 5, 'located_bikes_count': 3},
               {'id': 6, 'located_bikes_count': 4},
               {'id': 7, 'located_bikes_count': 3},
               {'id': 8, 'located_bikes_count': 3},
               {'id': 9, 'located_bikes_count': 3},
               {'id': 10, 'located_bikes_count': 3},
               {'id': 11, 'located_bikes_count': 4},
               {'id': 12, 'located_bikes_count': 4},
               {'id': 13, 'located_bikes_count': 2},
               {'id': 14, 'located_bikes_count': 3},
               {'id': 15, 'located_bikes_count': 3},
               {'id': 16, 'located_bikes_count': 3},
               {'id': 17, 'located_bikes_count': 4},
               {'id': 18, 'located_bikes_count': 4},
   

In [534]:
# make command
if len(MAX_LOC) != 0 and len(MIN_LOC) != 0:
    
    # 이미 bike 싣고 있음
    loaded_trucks = list(filter(lambda tr : tr['loaded_bikes_count'] >= 2, truck["trucks"]))
    
    # bike가 없는 경우
    unloaded_trucks = list(filter(lambda tr : tr['loaded_bikes_count'] < 2, truck["trucks"]))
    tmp_dict = {}
    for tr in unloaded_trucks:
        tr, loc = find_max_loc(tr, MAX_LOC, location, MEAN)
        if tr == -1:
            break
        
        load_num = location['locations'][loc]['located_bikes_count'] - MEAN
        command[tr['id']]["command"] += truck_move_plan(truck['trucks'][tr['id']]['location_id'], loc) \
                       + [5 for i in range(load_num)]
        truck['trucks'][tr['id']] = {'id': tr['id'], 'loaded_bikes_count': load_num, 'location_id': loc}
        location['locations'][loc]['located_bikes_count'] -= load_num
        loaded_trucks.append(truck['trucks'][tr['id']])
    
    
    # 배송    
    for dst in MIN_LOC[:5]:
        # 운행
        dst = dst[0]
        tr, v = find_truck(loaded_trucks, dst)
        src = tr['location_id']
        plan = truck_move_plan(src, dst)
        
        loaded_trucks.remove(tr)
        load_num = tr['loaded_bikes_count']
        command[tr['id']]["command"] += plan + [6 for i in range(load_num)]
        location['locations'][dst]['located_bikes_count'] += load_num
        truck['trucks'][tr['id']] = {'id': tr['id'], 'loaded_bikes_count': 0, 'location_id': dst}
        # 하차
    
    for cmd in command:
        cmd['command'] += [0 for i in range(10 - len(cmd['command']))]


[{'command': [5, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'truck_id': 0},
 {'command': [1, 1, 1, 1, 5, 0, 0, 0, 0, 0], 'truck_id': 1},
 {'command': [1, 1, 1, 5, 2, 2, 6, 0, 0, 0], 'truck_id': 2},
 {'command': [1, 1, 5, 0, 0, 0, 0, 0, 0, 0], 'truck_id': 3},
 {'command': [2, 2, 2, 2, 5, 1, 1, 1, 1, 6], 'truck_id': 4}]
{'trucks': [{'id': 0, 'loaded_bikes_count': 1, 'location_id': 0},
            {'id': 1, 'loaded_bikes_count': 1, 'location_id': 4},
            {'id': 2, 'loaded_bikes_count': 0, 'location_id': 13},
            {'id': 3, 'loaded_bikes_count': 1, 'location_id': 2},
            {'id': 4, 'loaded_bikes_count': 0, 'location_id': 24}]}
{'locations': [{'id': 0, 'located_bikes_count': 3},
               {'id': 1, 'located_bikes_count': 4},
               {'id': 2, 'located_bikes_count': 3},
               {'id': 3, 'located_bikes_count': 3},
               {'id': 4, 'located_bikes_count': 3},
               {'id': 5, 'located_bikes_count': 3},
               {'id': 6, 'located_bikes_count': 4

In [535]:
result = put_simulate(auth_key, {"commands": command})

flag = result["status"]

if flag == "finished":
    score = get_score(auth_key)
    print(score)
    break

200 | {"status":"ready","time":9,"failed_requests_count":"0.0","distance":"1.9"}


In [550]:
def find_max_loc(truck, MAX_LOC, location, MEAN):
    value = 100000
    answer = (-1, -1)
    for loc, k in MAX_LOC:
        if location['locations'][loc]['located_bikes_count'] > MEAN:
            v = loc - truck['location_id']
            x = v // 5
            y = v % 5
            for i in range(src + 1, src+y+1, 1 if src+y > src else -1):
                if i % 5 == 0:
                    x += 1
                    y += -5

            if abs(x) + abs(y) < value:
                value = abs(x) + abs(y)
                answer = (truck, loc)
    print(answer)
    return answer

In [548]:
a = [{'id': 0, 'loaded_bikes_count': 0, 'location_id': 0},
            {'id': 1, 'loaded_bikes_count': 0, 'location_id': 2},
            {'id': 2, 'loaded_bikes_count': 0, 'location_id': 3},
            {'id': 3, 'loaded_bikes_count': 0, 'location_id': 4},
            {'id': 4, 'loaded_bikes_count': 0, 'location_id': 5}]

def find_truck(trucks, loc):
    value = 100000
    answer = -1
    
    for tr in trucks:
        v = loc - tr['location_id']
        x = v // 5
        y = v % 5
        for i in range(src + 1, src+y+1, 1 if src+y > src else -1):
            if i % 5 == 0:
                x += 1
                y += -5
        
        if abs(x) + abs(y) == value:
            answer = answer if answer['loaded_bikes_count'] > tr['loaded_bikes_count'] else tr
        elif abs(x) + abs(y) < value:
            value = abs(x) + abs(y)
            answer = tr
            
    return answer, value

In [549]:
def truck_move_plan(src, dst):
    v = dst - src
    x = v // 5
    y = v % 5
    for i in range(src + 1, src+y+1, 1 if src+y > src else -1):
        if i % 5 == 0:
            x += 1
            y += -5
    
    plan = []
    for i in range(abs(x)):
        if x > 0:
            plan.append(2)
        else:
            plan.append(4)
    for i in range(abs(y)):
        if y > 0:
            plan.append(1)
        else:
            plan.append(3)
    
    return plan